In [334]:
import xml.etree.ElementTree as etree
archea=etree.parse('archea_1_3.svg')

In [335]:
#looking for group g19798
ns={'s':'http://www.w3.org/2000/svg'}
archroot=archea.getroot()
thegroup=archroot.findall(".//s:g[@id='g19798']",ns)[0]

In [336]:
trees=[g for g in archroot[3] if g.attrib['id'] != 'g19798']

In [337]:
archroot[3].get('transform')

'matrix(3.9820487,0,0,2.8817668,7204.3421,-32271.516)'

In [338]:
branchgroups= {g.attrib['id']:g for g in thegroup}
#branchgroups

In [339]:
import numpy as np
from vec2d import Vec2D
#####DIDN"T NOTICE<BUT the top group has a transform applied to it
grouptfm=Vec2D(*[float(s) for s in thegroup.get('transform')[len('translate('):-1].split(',')])
#AND WE MIGHT AS WELL APPLY THE TOP TFM AS WELL
#a,b,c,d,e,f=[float(s) for s in archroot[3].get('transform')[len('matrix('):-1].split(',')]
#MegaTfm=np.array([[a,c,e],[b,d,f],[0,0,1]])
def applyTfm(M,pt):
    return Vec2D(*np.array([*pt.car,1]).dot(M.T)[:-1])
def br_position(g):
    translate=None
    if 'transform' in g.attrib:
        translate=Vec2D(*[float(s) for s in g.attrib['transform'][10:-1].split(',')])
    path=g.find('./s:path',ns)
    #t=g.find('s:text',ns)
    #t.attrib['style']=t.attrib['style'][:t.attrib['style'].find('fill')+5]+'#f4476b'
    label=g.find('.//s:textPath',ns).text
    #print(label)
    a=path.attrib['style'].find('stroke-width:')+len('stroke-width:')
    b=path.attrib['style'][a:].find(';')
    if b==-1:
        width=int(path.get('style')[a:])
    else:
        width=int(path.attrib['style'][a:a+b])
    d=path.attrib['d']
    d=d.split(' ')
    r=[]
    for a in d:
        r.extend(a.split(','))#doing this cuz it's permitted to have c 0 0 3 5 or c 0,0 3,5
    # one of two things: m followed by relative path, or M followed by absolute

    absolute=False
    if 'M'==r.pop(0):
        absolute=True
    p0=Vec2D(float(r.pop(0)),float(r.pop(0)))
    if absolute:
        assert(r.pop(0)=='C')
    else:
        assert(r.pop(0)=='c')
    p1=Vec2D(float(r.pop(0)),float(r.pop(0)))
    p2=Vec2D(float(r.pop(0)),float(r.pop(0)))
    p3=Vec2D(float(r.pop(0)),float(r.pop(0)))
    #print('raw p0,p1,p2,p3:',p0,p1,p2,p3)
    if not absolute:
        p1=p1+p0
        p2=p2+p0
        p2=p2+p0
        p3=p3+p0
        #p3=p3+p2
    if translate:
        p0=p0+translate
        p1=p1+translate
        p2=p2+translate
        p3=p3+translate
    #important to apply in correct order, innermost to outermost
    p0=p0+grouptfm
    #p0=applyTfm(MegaTfm,p0)
    p1=p1+grouptfm
    #p1=applyTfm(MegaTfm,p1)
    p2=p2+grouptfm
    #p2=applyTfm(MegaTfm,p2)
    p3=p3+grouptfm
    #p3=applyTfm(MegaTfm,p3)
    #print('final p0-3:',p0,p1,p2,p3)
    return p0,p1,p2,p3
import numpy as np
def tr_position(t):
    m=[float(s) for s in t.get('transform')[len('matrix('):-1].split(',')]
    Mx=np.array([[m[0],m[2],m[4]],[m[1],m[3],m[5]],[0,0,1]])
    lines=t[0].findall('.s:line',ns)
    #both lines have same x1,yb=1,
    def v(i,a,b):
        return Vec2D(float(lines[i].get(a)),float(lines[i].get(b)))
    p0=v(0,'x1','y1')
    p1,p2=v(0,'x2','y2'),v(1,'x2','y2')
    p0=applyTfm(Mx,p0)
    #p0=applyTfm(MegaTfm,p0)
    p1=applyTfm(Mx,p1)
    #p1=applyTfm(MegaTfm,p1)
    p2=applyTfm(Mx,p2)
    #p2=applyTfm(MegaTfm,p2)
    theta=((p1-p0).t+(p2-p0).t)/2
    return p0,theta

In [340]:
poss_branches={}
for b in branchgroups:
    p0,p1,p2,p3 = br_position(branchgroups[b])
    poss_branches[b]={'p0':p0,'p1':p1,'p2':p2,'p3':p3}
poss_trees={}
for t in trees:
    p0,t0 = tr_position(t)
    poss_trees[t]={'p0':p0,'t0':t0}


In [341]:
#at this point, branches should be where the screen says they are
[(poss_branches[b],branchgroups[b].find('.//s:textPath',ns).text) for b in branchgroups]

[({'p0': Vec2D(x=446.73532,y=12371.997060000002),
   'p1': Vec2D(x=428.09413,y=12389.171380000002),
   'p2': Vec2D(x=883.6250600000001,y=12264.103210000001),
   'p3': Vec2D(x=408.76217,y=12463.248774000002)},
  None),
 ({'p0': Vec2D(x=-211.34491,y=12010.495270000001),
   'p1': Vec2D(x=-306.50561,y=11864.216520000002),
   'p2': Vec2D(x=-298.710789,y=11386.39232),
   'p3': Vec2D(x=-54.45769999999999,y=11797.292060000002)},
  None),
 ({'p0': Vec2D(x=-125.70005,y=12046.34301),
   'p1': Vec2D(x=-103.50026,y=12062.15481),
   'p2': Vec2D(x=-149.90047900000002,y=11553.243460000002),
   'p3': Vec2D(x=-3.7291618,y=12114.826640000001)},
  None),
 ({'p0': Vec2D(x=-363.87634,y=11980.778590000002),
   'p1': Vec2D(x=-417.1156900000001,y=11925.32429),
   'p2': Vec2D(x=482.3112899999999,y=10590.18029),
   'p3': Vec2D(x=-345.55323,y=11828.69029)},
  'Archæoglobales'),
 ({'p0': Vec2D(x=-602.17659,y=11948.276140000002),
   'p1': Vec2D(x=-693.94906,y=11943.674400000002),
   'p2': Vec2D(x=-938.73073,y=11085

In [342]:
[poss_trees[t] for t in trees]

[{'p0': Vec2D(x=-694.8130979683814,y=11878.681001921273),
  't0': -1.7228698102977966},
 {'p0': Vec2D(x=-901.799701670377,y=11907.168122934823),
  't0': -2.0901455608365844},
 {'p0': Vec2D(x=-960.2566821008363,y=12009.199284748698),
  't0': 0.19226587041050647},
 {'p0': Vec2D(x=-461.064619720851,y=11784.452784831436),
  't0': -1.6823530113409233},
 {'p0': Vec2D(x=-340.1881252430653,y=11826.315948906162),
  't0': -0.7518234188329591},
 {'p0': Vec2D(x=64.96429338570613,y=11769.058401031154),
  't0': -0.547026237325648},
 {'p0': Vec2D(x=-28.325691996758053,y=11606.631650511696),
  't0': -1.040366807496104},
 {'p0': Vec2D(x=-123.78842180610036,y=11583.406275248039),
  't0': -2.0176896528280803},
 {'p0': Vec2D(x=184.4455205792595,y=11952.017611395739),
  't0': -0.514633446651716},
 {'p0': Vec2D(x=180.2748517128962,y=12067.06980238413),
  't0': 0.45235013096960597},
 {'p0': Vec2D(x=617.5301428478871,y=12030.88978493318),
  't0': -0.3900156471850028},
 {'p0': Vec2D(x=560.4572740777617,y=11852

In [343]:
branch_from_point={}
p0points=[]
p3points=[]
for b in branchgroups:
    branch_from_point[poss_branches[b]['p0']]=branchgroups[b]
    p0points.append(poss_branches[b]['p0'])
    branch_from_point[poss_branches[b]['p3']]=branchgroups[b]
    p3points.append(poss_branches[b]['p3'])
for t in trees:
    #find the p0 or p3 that matches.
    phere=poss_trees[t]['p0']
    dist0,idx0=min([((phere-p).r,idx) for idx,p in enumerate(p0points)])
    dist3,idx3=min([((phere-p).r,idx) for idx,p in enumerate(p3points)])
    if dist0<dist3:
        p=p0points.pop(idx0)
        #we want to move branchpoints[p]['p0']
        g=branch_from_point[p]
        #goal=reverseTfm(MegaTfm,phere)
        path=g.find('s:path',ns)
        d=path.attrib['d']
        d=d.split(' ')
        r=[]
        for a in d:
            r.extend(a.split(','))
        r[1]=str(phere.car[0]-grouptfm)
        r[2]=str(goal.car[1]-grouptfm)
        r[0]='M'#so we don't have to convert back
        r[3]='C'
        if 'transform' in g.attrib:
            translate=Vec2D(*[float(s) for s in g.attrib['transform'][10:-1].split(',')])
            r[1]=str(float(r[1])-translate.car[0])
            r[2]=str(float(r[1])-translate.car[0])
        #r=[str(a) for a in r]
        path.attrib['d']=' '.join(r)
    else:#move branchpoints[p['p3]]
        p=p3points.pop(idx3)
        g=branch_from_point[p]
        #goal=reverseTfm(MegaTfm,phere)
        goal=goal-grouptfm
        if 'transform' in g.attrib:
            translate=Vec2D(*[float(s) for s in g.attrib['transform'][10:-1].split(',')])
            goal=goal-translate
        path=g.find('s:path',ns)
        d=path.attrib['d']
        d=d.split(' ')
        r=[]
        for a in d:
            r.extend(a.split(','))
        if r[0]=='M':
            r[-2]=str(goal.car[0])
            r[-1]=str(goal.car[1])
        else:
            r[-2]=str(goal.car[0]-float(r[1]))
            r[-1]=str(goal.car[1]-float(r[2]))
        path.attrib['d']=' '.join(r)

ValueError: could not convert string to float: 'Vec2D(x=6.6653775374451e-310,y=-12555.906)'

In [331]:
with open('modified_archaea.svg','w') as f:
    f.write(etree.tostring(archroot,encoding='unicode'))